# Set up an Open-source Agent via vLLM

### Requirement

Auto Agent setup need `vllm>=0.2.0`, you can use the following command to install the latest version of [vLLM](https://github.com/vllm-project/vllm).
```
pip install vllm
```

In [ ]:
!pip install vllm

In [ ]:
import autogen
import os

In [ ]:
def auto_agent_generation(
    model_name_or_hf_repo: str,
    world_size: int = 4,
    host: str = "127.0.0.1",
    port: str = "8000",
    logging_path: str = "logs/",
    seed: int = 42,
    temperature: int = 0,
):
    if "gpt-" in model_name_or_hf_repo:
        config = autogen.config_list_from_json(
            "OAI_CONFIG_LIST",
            filter_dict={
                "model": [model_name_or_hf_repo]
            },
        )
    else:
        model_name = model_name_or_hf_repo.split('/')[-1]
        config = [autogen.get_config_list(
            api_keys = ["EMPTY"],
            api_bases = [f"{host}:{port}"],
        )]
        # Use vllm to create a server with OpenAI API support
        os.system(f'nohup /root/miniconda3/envs/LLM/bin/python -m vllm.entrypoints.openai.api_server'
                  f' --host {host}'
                  f' --port {port}'
                  f' --model {model_name_or_hf_repo}'
                  f' --tensor-parallel-size {world_size} > "{logging_path}/serving_{model_name}.log" &')
    
    agent = autogen.AssistantAgent(
        name="assistant",
        llm_config={
            "seed": seed,  # seed for caching and reproducibility
            "config_list": config,  # a list of OpenAI API configurations
            "temperature": temperature,  # temperature for sampling
        },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
    )
    return agent